In [ ]:
library(ggplot2)
library(SingleR)
library(dplyr)
library(celldex)
library(RColorBrewer)
source("../tools/formating/formating.R")

In [ ]:
source("../tools/qc/seurat_qc.R")
path="/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"
results <- LoadSeuratMetaData(path)
results$srat

In [ ]:
qc_results <- RunSeuratQC(results$srat, '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat')
qc_results

In [ ]:
# library("here")
here()

In [ ]:
getwd()

In [ ]:
packageVersion("Seurat")

In [ ]:
current_path <- getwd()
print(current_path)

In [ ]:
here::here('api/tools/qc/scrublet_calls.tsv')

In [ ]:
source("../tools/qc/seurat_qc.R")
# path="/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
# adata_path = '/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad'
# path="/ps/ai-ready/data/tung/umi.h5Seurat"
path="/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
adata_path = 'ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad'
# adata_path = '/ps/ai-ready/data/tung/umi.h5ad'

results <- RunSeuratQC(path, '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', adata_path=adata_path, assay='originalexp')
results

In [ ]:
print(results)

In [ ]:
source("../tools/qc/seurat_qc.R")
# path="/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat"
# adata_path = '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad'
path="/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
adata_path = '/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad'

results <- RunSeuratQC(path, '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', unique_id='Seurat test', adata_path=adata_path)
results

In [ ]:
# srat <- LoadSeurat('/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.Robj')
# srat <- LoadSeurat("/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat")
srat <- LoadSeurat("/ps/ai-ready/data/tung/umi.h5Seurat")
# srat@reductions <- NULL
srat

In [ ]:
DefaultAssay(srat) <- "originalexp"
srat

In [ ]:
# remotes::install_github('chris-mcginnis-ucsf/DoubletFinder')

In [ ]:
srat <- NormalizeData(srat)
srat <- FindVariableFeatures(srat, selection.method="vst", nfeatures=2000)
srat <- ScaleData(srat)
srat <- RunPCA(srat)
srat <- RunUMAP(srat, dims=1:10)

In [ ]:
srat

In [ ]:
srat@meta.data

In [ ]:
library(DoubletFinder)
set.seed(123)
sweep.res.list <- paramSweep(srat, PCs=1:10, sct=FALSE)
sweep.stats <- summarizeSweep(sweep.res.list, GT=FALSE)
bcmvn <- find.pK(sweep.stats)

In [ ]:
packageVersion("DoubletFinder")

In [ ]:
nExp_poi <- round(0.075*nrow(srat@meta.data)) 

In [ ]:
srat <- doubletFinder(srat, PCs = 1:10, pN = 0.25, pK = 0.09, nExp = nExp_poi, reuse.pANN = FALSE, sct = FALSE)

In [ ]:
library(stringr)
colnames(srat@meta.data)[str_starts(colnames(srat@meta.data),"pANN_")] <- "Double_score"
colnames(srat@meta.data)[str_starts(colnames(srat@meta.data),"DF.classifications_")] <- "Is_Double"

In [ ]:
srat <- FindNeighbors(srat, dims=1:10)
srat <- FindClusters(srat, resolution=0.5)

In [ ]:
srat@meta.data

In [ ]:
sce <- as.SingleCellExperiment(srat)
sce

In [ ]:
# BiocManager::install("plger/scDblfinder")
# install.packages("irlba",type = "source")
remotes::install_github("Moonerss/scrubletR")

In [ ]:
scrublet_R <- function(seurat_obj, python_home = Sys.which("python3"),
                       return_results_only = FALSE, min_counts=2,
                       min_cells=3, expected_doublet_rate=0.06,
                       min_gene_variability_pctl=85,
                       n_prin_comps=50, sim_doublet_ratio=2, n_neighbors=NULL) {
  ## use the python
  reticulate::use_python(python_home)

  ## test whether have the modules
  if (!reticulate::py_module_available("scrublet")) {
    stop("python module scrublet does not seem to be installed; - try running 'py_config()'")
  }

  ## source .py file
  reticulate::source_python(paste(system.file(package = "scrubletR"), "scrublet.py", sep = "/"))

  ## prepare the data
  X <- as(Matrix::t(seurat_obj@assays$RNA@counts), "TsparseMatrix")
  i <- as.integer(X@i)
  j <- as.integer(X@j)
  val <- X@x
  dim <- as.integer(X@Dim)
  if(is.null(n_neighbors)){
    n_neighbors<-round(0.5*sqrt(nrow(X)))
  }

  ## do the scrublet analysis
  scrublet_py_args<-c(list(i=i, j=j, val=val, dim=dim,
                           expected_doublet_rate=expected_doublet_rate, min_counts=min_counts,
                           min_cells=min_cells, min_gene_variability_pctl=min_gene_variability_pctl, n_prin_comps=n_prin_comps,
                           sim_doublet_ratio=sim_doublet_ratio, n_neighbors=n_neighbors))
  scrublet_res <- do.call(scrublet_py, scrublet_py_args)
  names(scrublet_res)<-c("doublet_scores", "predicted_doublets")

  ## get the result
  if (return_results_only) {
    return(scrublet_res)
  } else {
    seurat_obj[["doublet_scores"]] <- scrublet_res$doublet_scores
    seurat_obj[["predicted_doublets"]] <- scrublet_res$predicted_doublets
    return(seurat_obj)
  }
}

In [ ]:
# library(scrubletR)
res <- scrublet_R(srat)
res

In [ ]:
sce <- runScrublet(sce)
sce

In [ ]:
sce <- scDblFinder(sce, nfeatures=2000, dims=30, iter=5)

In [ ]:
doublet_annnotation <- AnnotateDroplet(srat[["RNA"]]@counts)

In [ ]:
doublet_annnotation

In [ ]:
adata_path <- Convert('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', dest = "h5ad" , overwrite = TRUE)
adata_path

In [ ]:
path="/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"

srat <- LoadSeurat(path)

srat
srat@version

In [ ]:
Project(srat)

In [ ]:
obj <- readRDS("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj")
obj

In [ ]:
srat_v2 <- get(load(path))
srat_v2

In [ ]:
class(srat_v2) == 'seurat'

In [ ]:
srat_v2@meta.data

In [ ]:
srat[['RNA']]@counts

In [ ]:
v4_obj <- CreateSeuratObject(counts=srat[['RNA']]@counts, meta.data=srat@meta.data)
v4_obj

In [ ]:
SaveH5Seurat(v4_obj, filename='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5seurat', overwrite=TRUE)

In [ ]:
Cells(v4_obj)

In [ ]:
v4_obj@meta.data

In [ ]:
v4_obj[['RNA']]@var.features

In [ ]:
srat@reductions$pca@misc <- list()
srat@reductions$tsne@misc <- list()
# srat@reductions$umap@misc <- list()

In [ ]:
SaveH5Seurat(srat, filename='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5seurat', overwrite=TRUE)

In [ ]:
saveRDS(srat, "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds")

In [ ]:
srat1 <- readRDS(file="/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds")
srat1

In [ ]:
as.h5Seurat(srat1, '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5seurat', overwrite = TRUE, verbose = TRUE)

In [ ]:
SaveH5Seurat(srat1, filename='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5seurat', overwrite=TRUE)

In [ ]:
adata_path <- Convert('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds', dest = "h5ad" , overwrite = TRUE)

In [ ]:
class(srat1) == 'Seurat'

In [ ]:
v4_obj@version

In [ ]:
srat2 <- readRDS(file="/ps/ai-ready/data/tung/tung.rds")
srat2

In [ ]:
class(srat2)

In [ ]:
# The [[ operator can add columns to object metadata. This is a great place to stash QC stats
srat[["percent.mt"]] <- PercentageFeatureSet(srat, pattern = "^MT-")

In [ ]:
# Visualize QC metrics as a violin plot
VlnPlot(srat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)

In [ ]:
plot1 <- FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2

In [ ]:
srat <- subset(srat, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)

In [ ]:
srat <- NormalizeData(srat, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
srat <- FindVariableFeatures(srat, selection.method = "vst")
srat <- ScaleData(srat, features = rownames(srat))

In [ ]:
srat <- RunPCA(srat, features = VariableFeatures(srat), ndims.print = 6:10, nfeatures.print = 10)

In [ ]:
DimHeatmap(srat, dims = c(8, 10))

In [ ]:
srat <- CellCycleScoring(srat, g2m.features = cc.genes$g2m.genes, s.features = cc.genes$s.genes)

In [ ]:
# Read in the expression matrix The first row is a header row, the first column is rownames
exp.mat <- read.table(file = "../tools/qc/nestorawa_forcellcycle_expressionMatrix.txt", header = TRUE,
    as.is = TRUE, row.names = 1)

# A list of cell cycle markers, from Tirosh et al, 2015, is loaded with Seurat.  We can
# segregate this list into markers of G2/M phase and markers of S phase
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes

In [ ]:
srat <- CellCycleScoring(srat, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)
# view cell cycle scores and phase assignments
head(srat[[]])

In [ ]:
VlnPlot(srat, features = c("S.Score", "G2M.Score"), group.by = "orig.ident",
    ncol = 4, pt.size = 0.1)

In [ ]:
srat <- ScaleData(srat, vars.to.regress = c("S.Score", "G2M.Score"), features = rownames(srat))
srat

In [ ]:
remotes::install_github('chris-mcginnis-ucsf/DoubletFinder')

In [ ]:
# define the expected number of doublet cellscells.
nExp <- round(ncol(srat) * 0.04)  # expect 4% doublets
srat <- doubletFinder_v3(srat, pN = 0.25, pK = 0.09, nExp = nExp, PCs = 1:10)

In [ ]:
rm(srat)
gc()